In [1]:
import numpy as np
import os
from numba import jit

In [13]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [9]:
path = './data'
files = os.listdir(path)


In [14]:
for i, f in enumerate(files):
    print(i, f)

0 ks_10000_0
1 ks_1000_0
2 ks_100_0
3 ks_100_1
4 ks_100_2
5 ks_106_0
6 ks_19_0
7 ks_200_0
8 ks_200_1
9 ks_300_0
10 ks_30_0
11 ks_400_0
12 ks_40_0
13 ks_45_0
14 ks_4_0
15 ks_500_0
16 ks_50_0
17 ks_50_1
18 ks_60_0
19 ks_82_0
20 ks_lecture_dp_1
21 ks_lecture_dp_2


<IPython.core.display.Javascript object>

In [54]:
def parse_data(data):
    lines = data.split("\n")
    n, cap = lines[0].split()
    data = [line.split() for line in lines if len(line) > 0]
    n, cap = data.pop(0)
    return np.int(n), np.float(cap), np.array(data, dtype=float)

<IPython.core.display.Javascript object>

In [86]:
file = files[14]
with open(os.path.join(path, file)) as fp:
    data = fp.read()

n, K, data = parse_data(data)
values = data[:, 0]
weights = data[:, 1]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [152]:
def check_results(selection, weights, values, cap):
    assert (
        selection * weights
    ).sum() <= cap, f"Too heavy!: The capacity is {cap} but the total weight is {(selection * weights).sum()}."


def test_method(func, file):
    with open(os.path.join(path, file)) as fp:
        data = fp.read()
    n, K, data = parse_data(data)
    values = data[:, 0]
    weights = data[:, 1]
    print(f"Total number: {n} \nCapacity: {K}")
    selection, weight, count, spare = func(values, weights, K)
    final_value = (selection * values).sum()
    print(
        f"Final Weight: {weight} \nFinal Value: {final_value}\nNumber of Items: {count}\nSpare capacity: {spare}"
    )

<IPython.core.display.Javascript object>

In [188]:
def greedy_dens(values, weights, capacity):
    n = len(values)
    densities = values / weights
    idx = np.argsort(densities)
    ks_weight = 0
    selection = np.zeros(n)
    for i in reversed(idx):
        selection[i] = 1
        if np.sum(selection * weights) > capacity:
            selection[i] = 0
            break
    final_weight = np.sum(selection * weights)
    spare = capacity - final_weight
    return (
        selection.astype(int),
        np.sum(selection * weights).astype(int),
        np.sum(selection).astype(int),
        spare,
    )


def greedy_dens2(values, weights, capacity):
    n = len(values)
    densities = values / weights
    idx = np.argsort(densities)
    ks_weight = 0
    selection = np.zeros(n)
    for i in reversed(idx):
        selection[i] = 1
        if np.sum(selection * weights) > capacity:
            selection[i] = 0
            continue
    final_weight = np.sum(selection * weights)
    spare = capacity - final_weight
    return (
        selection.astype(int),
        np.sum(selection * weights).astype(int),
        np.sum(selection).astype(int),
        spare,
    )


@jit
def DynPj(values, weights, capacity):
    rows, cols = int(capacity) + 1, len(values) + 1
    mat = np.zeros((rows, cols))
    for row in range(rows):
        for col in range(1, cols):

            i = col - 1
            wi = int(weights[i])
            vi = int(values[i])
            if row < wi:
                mat[row, col] = mat[row, col - 1]
            else:
                mat[row, col] = max(mat[row, col - 1], vi + mat[row - wi, col - 1])
    selection = np.zeros(len(values), dtype=int)
    row = rows - 1
    for c in range(len(values), 0, -1):
        if mat[row, c] != mat[row, c - 1]:
            selection[c - 1] = 1
            row = row - int(weights[c - 1])

    #     return mat[-1, -1], selection
    final_weight = np.sum(selection * weights)
    final_value = np.sum(selection * values)
    #     if abs(final_value - mat[-1, -1]) > 1e-3:
    #         print(f"***Warning: {np.sum(selection * values)} =/= {mat[-1,-1]}")
    spare = capacity - final_weight
    return (
        selection.astype(int),
        np.sum(selection * weights).astype(int),
        np.sum(selection).astype(int),
        spare,
    )


def DynP(values, weights, capacity):
    rows, cols = int(capacity) + 1, len(values) + 1
    mat = np.zeros((rows, cols))
    for row in range(rows):
        for col in range(1, cols):

            i = col - 1
            wi = int(weights[i])
            vi = int(values[i])
            if row < wi:
                mat[row, col] = mat[row, col - 1]
            else:
                mat[row, col] = max(mat[row, col - 1], vi + mat[row - wi, col - 1])
    selection = np.zeros(len(values), dtype=int)
    row = rows - 1
    for c in range(len(values), 0, -1):
        if mat[row, c] != mat[row, c - 1]:
            selection[c - 1] = 1
            row = row - int(weights[c - 1])

    #     return mat[-1, -1], selection
    final_weight = np.sum(selection * weights)
    final_value = np.sum(selection * values)
    #     if abs(final_value - mat[-1, -1]) > 1e-3:
    #         print(f"***Warning: {np.sum(selection * values)} =/= {mat[-1,-1]}")
    spare = capacity - final_weight
    return (
        selection.astype(int),
        np.sum(selection * weights).astype(int),
        np.sum(selection).astype(int),
        spare,
    )

def LinearRelaxation(values, weights, capacity):
    n = len(values)
    densities = values / weights
    idx = np.argsort(densities)
    ks_weight = 0
    selection = np.zeros(n)
    for i in reversed(idx):
        selection[i] = 1
        if np.sum(selection * weights) > capacity:
            selection[i] = 0
            final_weight = (selection*weights).sum()
            spare = capacity-final_weight
            xi = spare/weights[i]
            final_value = (selection*values).sum()+xi*values[i]
            return final_value


<IPython.core.display.Javascript object>

In [192]:
np.binary_repr(12)

'1100'

<IPython.core.display.Javascript object>

In [189]:
file = files[13]
with open(os.path.join(path, file)) as fp:
    data = fp.read()

n, K, data = parse_data(data)
values = data[:, 0]
weights = data[:, 1]
LinearRelaxation(values, weights, K)

24265.553963971113

<IPython.core.display.Javascript object>

In [187]:
%%time
test_method(DynPj, files[7])

Total number: 200 
Capacity: 100000.0
Final Weight: 99993 
Final Value: 100236.0
Number of Items: 4
Spare capacity: 7.0
Wall time: 163 ms


<IPython.core.display.Javascript object>

In [181]:
%%time
test_method(DynP, files[7])

Total number: 200 
Capacity: 100000.0
Final Weight: 99993 
Final Value: 100236.0
Number of Items: 4
Spare capacity: 7.0
Wall time: 22.3 s


<IPython.core.display.Javascript object>

In [183]:
%%time
test_method(greedy_dens2, files[11])

Total number: 400 
Capacity: 9486367.0
Final Weight: 9485926 
Final Value: 3966813.0
Number of Items: 7
Spare capacity: 441.0
Wall time: 14 ms


<IPython.core.display.Javascript object>

In [129]:
list(range(len(values), 0, -1))

[4, 3, 2, 1]

<IPython.core.display.Javascript object>

In [168]:
for i, f in enumerate(files):
    print(i, f)

0 ks_10000_0
1 ks_1000_0
2 ks_100_0
3 ks_100_1
4 ks_100_2
5 ks_106_0
6 ks_19_0
7 ks_200_0
8 ks_200_1
9 ks_300_0
10 ks_30_0
11 ks_400_0
12 ks_40_0
13 ks_45_0
14 ks_4_0
15 ks_500_0
16 ks_50_0
17 ks_50_1
18 ks_60_0
19 ks_82_0
20 ks_lecture_dp_1
21 ks_lecture_dp_2


<IPython.core.display.Javascript object>

In [172]:
class Node():
    def __init__(self,weight,value,level):
        self.w = weight
        self.v = value
        self.l = level
    def children(self):
        level = self.l + 1
        self.left = Node(weight[self])

<IPython.core.display.Javascript object>

In [205]:
class Tree:
    def __init__(self, weights, values, capacity):
        densities = values / weights
        idx = np.argsort(densities)
        self.weights = weights[idx]
        self.values = values[idx]
        self.cap = capacity
        self.active_level = 0
        self.count = len(weights)
        self.selection = np.ones(self.count) * -1
        self.best_selection = None
        self.best_value = 0
    @property
    def get_lr(self):
        weight = self.get_weight()
        return LinearRelaxation(self.values[l:], self.weights[l:], self.cap - weight)

    @property
    def get_value(self):
        idx = self.selection > -1
        return (self.values[idx] * self.selection[idx]).sum()

    @property
    def get_weight(self):
        idx = self.selection > -1
        return (self.weights[idx] * self.selection[idx]).sum()

    @property
    def get_room(self):
        return self.cap - self.get_weight

    @property
    def is_end(self):
        return self.selection[-1] == -1

    @property
    def get_next_level(self):
        return np.argmin(self.selection)

    def step(self):
        estim = self.get_lr
        room = self.cap
        value = self.get_value
        for s in [1,0]:
            i = self.get_next_level
            self.selection[i]=s
            self.

<IPython.core.display.Javascript object>

In [200]:
class test:
    def __init__(self):
        pass

    @property
    def rand(self):
        return np.random.rand()

<IPython.core.display.Javascript object>

In [201]:
t = test()

<IPython.core.display.Javascript object>

In [204]:
t.rand

0.5540102778304988

<IPython.core.display.Javascript object>